In [1]:
import numpy as np
import time

In [2]:
class Node():
    def __init__(self,state,parent,action,depth,step_cost,path_cost,heuristic_cost):
        self.state = state 
        self.parent = parent # parent node
        self.action = action # move up, left, down, right
        self.depth = depth # depth of the node in the tree
        self.step_cost = step_cost # g(n), the cost to take the step
        self.path_cost = path_cost # accumulated g(n), the cost to reach the current node
        self.heuristic_cost = heuristic_cost # h(n), cost to reach goal state from the current node
        
        # children node
        self.move_up = None 
        self.move_left = None
        self.move_down = None
        self.move_right = None
    
    # see if moving down is valid
    def try_move_down(self):
        # index of the empty tile
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[0] == 0:
            return False
        else:
            up_value = self.state[zero_index[0]-1,zero_index[1]] # value of the upper tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = up_value
            new_state[zero_index[0]-1,zero_index[1]] = 0
            return new_state,up_value
        # see if moving right is valid
    def try_move_right(self):
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[1] == 0:
            return False
        else:
            left_value = self.state[zero_index[0],zero_index[1]-1] # value of the left tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = left_value
            new_state[zero_index[0],zero_index[1]-1] = 0
            return new_state,left_value
         # see if moving up is valid
    def try_move_up(self):
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[0] == 2:
            return False
        else:
            lower_value = self.state[zero_index[0]+1,zero_index[1]] # value of the lower tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = lower_value
            new_state[zero_index[0]+1,zero_index[1]] = 0
            return new_state,lower_value
         # see if moving left is valid
    def try_move_left(self):
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[1] == 2:
            return False
        else:
            right_value = self.state[zero_index[0],zero_index[1]+1] # value of the right tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = right_value
            new_state[zero_index[0],zero_index[1]+1] = 0
            return new_state,right_value
        # return user specified heuristic cost
    def get_h_cost(self,new_state,goal_state,heuristic_function,path_cost,depth):
        if heuristic_function == 'num_misplaced':
            return self.h_misplaced_cost(new_state,goal_state)
        elif heuristic_function == 'manhattan':
            return self.h_manhattan_cost(new_state,goal_state)
        # since this game is made unfair by setting the step cost as the value of the tile being moved
        # to make it fair, I made all the step cost as 1
        # made it a best-first-search with manhattan heuristic function
        elif heuristic_function == 'fair_manhattan':
            return self.h_manhattan_cost(new_state,goal_state) - path_cost + depth
        # return heuristic cost: number of misplaced tiles
    def h_misplaced_cost(self,new_state,goal_state):
        cost = np.sum(new_state != goal_state)-1 # minus 1 to exclude the empty tile
        if cost > 0:
            return cost
        else:
            return 0 # when all tiles matches
         # return heuristic cost: sum of Manhattan distance to reach the goal state
    def h_manhattan_cost(self,new_state,goal_state):
        current = new_state
        # digit and coordinates they are supposed to be
        goal_position_dic = {1:(0,0),2:(0,1),3:(0,2),8:(1,0),0:(1,1),4:(1,2),7:(2,0),6:(2,1),5:(2,2)} 
        sum_manhattan = 0
        for i in range(3):
            for j in range(3):
                if current[i,j] != 0:
                    sum_manhattan += sum(abs(a-b) for a,b in zip((i,j), goal_position_dic[current[i,j]]))
        return sum_manhattan
    # once the goal node is found, trace back to the root node and print out the path
    def print_path(self):
        # create FILO stacks to place the trace
        state_trace = [self.state]
        action_trace = [self.action]
        depth_trace = [self.depth]
        step_cost_trace = [self.step_cost]
        path_cost_trace = [self.path_cost]
        heuristic_cost_trace = [self.heuristic_cost]
        
        # add node information as tracing back up the tree
        while self.parent:
            self = self.parent
            state_trace.append(self.state)
            action_trace.append(self.action)
            depth_trace.append(self.depth)
            step_cost_trace.append(self.step_cost)
            path_cost_trace.append(self.path_cost)
            heuristic_cost_trace.append(self.heuristic_cost)

        # print out the path
        step_counter = 0
        while state_trace:
            print ('step',step_counter)
            print (state_trace.pop())
            print ('action=',action_trace.pop(),', depth=',str(depth_trace.pop()),\
            ', step cost=',str(step_cost_trace.pop()),', total_cost=',\
            str(path_cost_trace.pop() + heuristic_cost_trace.pop()),'\n')
            
            step_counter += 1

    def depth_first_search(self, goal_state):
        start = time.time()
        queue = [self] # queue of found but unvisited nodes, FILO
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [0] # queue of node depth
        path_cost_queue = [0] # queue for path cost
        visited = set([]) # record visited states
        
        while queue:
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0) # select and remove the first node in the queue
            queue_num_nodes_popped += 1 
            
            current_depth = depth_queue.pop(0) # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0) # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.state.reshape(1,9)[0])) # add state, which is represented as a tuple
            
            # when the goal state is found, trace back to the root node and print out the path
            if np.array_equal(current_node.state,goal_state):
                current_node.print_path()
                
                print( 'Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                print ('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                print ('Time spent: %0.2fs' % (time.time()-start))
                return True
            else:                
                # see if moving upper tile down is a valid move
                if current_node.try_move_down():
                    new_state,up_value = current_node.try_move_down()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_down = Node(state=new_state,parent=current_node,action='down',depth=current_depth+1,\
                                              step_cost=up_value,path_cost=current_path_cost+up_value,heuristic_cost=0)
                        queue.insert(0,current_node.move_down)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+up_value)
                    
                # see if moving left tile to the right is a valid move
                if current_node.try_move_right():
                    new_state,left_value = current_node.try_move_right()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_right = Node(state=new_state,parent=current_node,action='right',depth=current_depth+1,\
                                              step_cost=left_value,path_cost=current_path_cost+left_value,heuristic_cost=0)
                        queue.insert(0,current_node.move_right)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+left_value)
                # see if moving lower tile up is a valid move
                if current_node.try_move_up():
                    new_state,lower_value = current_node.try_move_up()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_up = Node(state=new_state,parent=current_node,action='up',depth=current_depth+1,\
                                              step_cost=lower_value,path_cost=current_path_cost+lower_value,heuristic_cost=0)
                        queue.insert(0,current_node.move_up)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+lower_value)

                # see if moving right tile to the left is a valid move
                if current_node.try_move_left():
                    new_state,right_value = current_node.try_move_left()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_left = Node(state=new_state,parent=current_node,action='left',depth=current_depth+1,\
                                              step_cost=right_value,path_cost=current_path_cost+right_value,heuristic_cost=0)
                        queue.insert(0,current_node.move_left)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+right_value)
                        
            
           
        

In [3]:
    
        
   test = np.array([1,2,3,8,6,4,7,5,0]).reshape(3,3)
easy = np.array([1,3,4,8,6,2,7,0,5]).reshape(3,3)
medium = np.array([2,8,1,0,4,3,7,6,5]).reshape(3,3)
hard = np.array([5,6,7,4,0,8,3,2,1]).reshape(3,3)

initial_state = test
goal_state = np.array([1,2,3,8,0,4,7,6,5]).reshape(3,3)
print(initial_state,'\n')


[[1 2 3]
 [8 6 4]
 [7 5 0]] 



In [4]:
print(goal_state)


[[1 2 3]
 [8 0 4]
 [7 6 5]]


In [5]:
root_node = Node(state=initial_state,parent=None,action=None,depth=0,step_cost=0,path_cost=0,heuristic_cost=0)
            

In [6]:
 # search as far as possible along each branch before backtracking, using stack
root_node.depth_first_search(goal_state)

step 0
[[1 2 3]
 [8 6 4]
 [7 5 0]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[1 2 3]
 [8 6 4]
 [7 0 5]]
action= right , depth= 1 , step cost= 5 , total_cost= 5 

step 2
[[1 2 3]
 [8 6 4]
 [0 7 5]]
action= right , depth= 2 , step cost= 7 , total_cost= 12 

step 3
[[1 2 3]
 [0 6 4]
 [8 7 5]]
action= down , depth= 3 , step cost= 8 , total_cost= 20 

step 4
[[1 2 3]
 [6 0 4]
 [8 7 5]]
action= left , depth= 4 , step cost= 6 , total_cost= 26 

step 5
[[1 2 3]
 [6 4 0]
 [8 7 5]]
action= left , depth= 5 , step cost= 4 , total_cost= 30 

step 6
[[1 2 3]
 [6 4 5]
 [8 7 0]]
action= up , depth= 6 , step cost= 5 , total_cost= 35 

step 7
[[1 2 3]
 [6 4 5]
 [8 0 7]]
action= right , depth= 7 , step cost= 7 , total_cost= 42 

step 8
[[1 2 3]
 [6 4 5]
 [0 8 7]]
action= right , depth= 8 , step cost= 8 , total_cost= 50 

step 9
[[1 2 3]
 [0 4 5]
 [6 8 7]]
action= down , depth= 9 , step cost= 6 , total_cost= 56 

step 10
[[1 2 3]
 [4 0 5]
 [6 8 7]]
action= left , depth= 10 , step co


step 131
[[6 2 3]
 [1 8 0]
 [4 7 5]]
action= down , depth= 131 , step cost= 5 , total_cost= 671 

step 132
[[6 2 3]
 [1 0 8]
 [4 7 5]]
action= right , depth= 132 , step cost= 8 , total_cost= 679 

step 133
[[6 2 3]
 [1 7 8]
 [4 0 5]]
action= up , depth= 133 , step cost= 7 , total_cost= 686 

step 134
[[6 2 3]
 [1 7 8]
 [4 5 0]]
action= left , depth= 134 , step cost= 5 , total_cost= 691 

step 135
[[6 2 3]
 [1 7 0]
 [4 5 8]]
action= down , depth= 135 , step cost= 8 , total_cost= 699 

step 136
[[6 2 3]
 [1 0 7]
 [4 5 8]]
action= right , depth= 136 , step cost= 7 , total_cost= 706 

step 137
[[6 2 3]
 [1 5 7]
 [4 0 8]]
action= up , depth= 137 , step cost= 5 , total_cost= 711 

step 138
[[6 2 3]
 [1 5 7]
 [4 8 0]]
action= left , depth= 138 , step cost= 8 , total_cost= 719 

step 139
[[6 2 3]
 [1 5 0]
 [4 8 7]]
action= down , depth= 139 , step cost= 7 , total_cost= 726 

step 140
[[6 2 3]
 [1 0 5]
 [4 8 7]]
action= right , depth= 140 , step cost= 5 , total_cost= 731 

step 141
[[6 2 3]
 [

step 246
[[2 3 8]
 [6 0 5]
 [1 7 4]]
action= left , depth= 246 , step cost= 6 , total_cost= 1244 

step 247
[[2 3 8]
 [6 7 5]
 [1 0 4]]
action= up , depth= 247 , step cost= 7 , total_cost= 1251 

step 248
[[2 3 8]
 [6 7 5]
 [1 4 0]]
action= left , depth= 248 , step cost= 4 , total_cost= 1255 

step 249
[[2 3 8]
 [6 7 0]
 [1 4 5]]
action= down , depth= 249 , step cost= 5 , total_cost= 1260 

step 250
[[2 3 8]
 [6 0 7]
 [1 4 5]]
action= right , depth= 250 , step cost= 7 , total_cost= 1267 

step 251
[[2 3 8]
 [6 4 7]
 [1 0 5]]
action= up , depth= 251 , step cost= 4 , total_cost= 1271 

step 252
[[2 3 8]
 [6 4 7]
 [1 5 0]]
action= left , depth= 252 , step cost= 5 , total_cost= 1276 

step 253
[[2 3 8]
 [6 4 0]
 [1 5 7]]
action= down , depth= 253 , step cost= 7 , total_cost= 1283 

step 254
[[2 3 8]
 [6 0 4]
 [1 5 7]]
action= right , depth= 254 , step cost= 4 , total_cost= 1287 

step 255
[[2 3 8]
 [0 6 4]
 [1 5 7]]
action= right , depth= 255 , step cost= 6 , total_cost= 1293 

step 256
[[

action= up , depth= 370 , step cost= 4 , total_cost= 1818 

step 371
[[3 8 5]
 [2 7 4]
 [6 0 1]]
action= right , depth= 371 , step cost= 1 , total_cost= 1819 

step 372
[[3 8 5]
 [2 7 4]
 [0 6 1]]
action= right , depth= 372 , step cost= 6 , total_cost= 1825 

step 373
[[3 8 5]
 [0 7 4]
 [2 6 1]]
action= down , depth= 373 , step cost= 2 , total_cost= 1827 

step 374
[[3 8 5]
 [7 0 4]
 [2 6 1]]
action= left , depth= 374 , step cost= 7 , total_cost= 1834 

step 375
[[3 8 5]
 [7 4 0]
 [2 6 1]]
action= left , depth= 375 , step cost= 4 , total_cost= 1838 

step 376
[[3 8 5]
 [7 4 1]
 [2 6 0]]
action= up , depth= 376 , step cost= 1 , total_cost= 1839 

step 377
[[3 8 5]
 [7 4 1]
 [2 0 6]]
action= right , depth= 377 , step cost= 6 , total_cost= 1845 

step 378
[[3 8 5]
 [7 4 1]
 [0 2 6]]
action= right , depth= 378 , step cost= 2 , total_cost= 1847 

step 379
[[3 8 5]
 [0 4 1]
 [7 2 6]]
action= down , depth= 379 , step cost= 7 , total_cost= 1854 

step 380
[[3 8 5]
 [4 0 1]
 [7 2 6]]
action= le

action= right , depth= 517 , step cost= 1 , total_cost= 2400 

step 518
[[0 8 5]
 [3 1 7]
 [2 6 4]]
action= down , depth= 518 , step cost= 3 , total_cost= 2403 

step 519
[[8 0 5]
 [3 1 7]
 [2 6 4]]
action= left , depth= 519 , step cost= 8 , total_cost= 2411 

step 520
[[8 5 0]
 [3 1 7]
 [2 6 4]]
action= left , depth= 520 , step cost= 5 , total_cost= 2416 

step 521
[[8 5 7]
 [3 1 0]
 [2 6 4]]
action= up , depth= 521 , step cost= 7 , total_cost= 2423 

step 522
[[8 5 7]
 [3 1 4]
 [2 6 0]]
action= up , depth= 522 , step cost= 4 , total_cost= 2427 

step 523
[[8 5 7]
 [3 1 4]
 [2 0 6]]
action= right , depth= 523 , step cost= 6 , total_cost= 2433 

step 524
[[8 5 7]
 [3 1 4]
 [0 2 6]]
action= right , depth= 524 , step cost= 2 , total_cost= 2435 

step 525
[[8 5 7]
 [0 1 4]
 [3 2 6]]
action= down , depth= 525 , step cost= 3 , total_cost= 2438 

step 526
[[8 5 7]
 [1 0 4]
 [3 2 6]]
action= left , depth= 526 , step cost= 1 , total_cost= 2439 

step 527
[[8 5 7]
 [1 4 0]
 [3 2 6]]
action= lef

...

action= down , depth= 7841 , step cost= 2 , total_cost= 35588 

step 7842
[[4 5 8]
 [7 0 6]
 [1 3 2]]
action= right , depth= 7842 , step cost= 6 , total_cost= 35594 

step 7843
[[4 5 8]
 [7 3 6]
 [1 0 2]]
action= up , depth= 7843 , step cost= 3 , total_cost= 35597 

step 7844
[[4 5 8]
 [7 3 6]
 [1 2 0]]
action= left , depth= 7844 , step cost= 2 , total_cost= 35599 

step 7845
[[4 5 8]
 [7 3 0]
 [1 2 6]]
action= down , depth= 7845 , step cost= 6 , total_cost= 35605 

step 7846
[[4 5 8]
 [7 0 3]
 [1 2 6]]
action= right , depth= 7846 , step cost= 3 , total_cost= 35608 

step 7847
[[4 5 8]
 [0 7 3]
 [1 2 6]]
action= right , depth= 7847 , step cost= 7 , total_cost= 35615 

step 7848
[[4 5 8]
 [1 7 3]
 [0 2 6]]
action= up , depth= 7848 , step cost= 1 , total_cost= 35616 

step 7849
[[4 5 8]
 [1 7 3]
 [2 0 6]]
action= left , depth= 7849 , step cost= 2 , total_cost= 35618 

step 7850
[[4 5 8]
 [1 7 3]
 [2 6 0]]
action= left , depth= 7850 , step cost= 6 , total_cost= 35624 

step 7851
[[4 5 8]


 [7 2 6]]
action= right , depth= 7988 , step cost= 3 , total_cost= 36200 

step 7989
[[5 8 1]
 [0 4 3]
 [7 2 6]]
action= right , depth= 7989 , step cost= 4 , total_cost= 36204 

step 7990
[[5 8 1]
 [7 4 3]
 [0 2 6]]
action= up , depth= 7990 , step cost= 7 , total_cost= 36211 

step 7991
[[5 8 1]
 [7 4 3]
 [2 0 6]]
action= left , depth= 7991 , step cost= 2 , total_cost= 36213 

step 7992
[[5 8 1]
 [7 4 3]
 [2 6 0]]
action= left , depth= 7992 , step cost= 6 , total_cost= 36219 

step 7993
[[5 8 1]
 [7 4 0]
 [2 6 3]]
action= down , depth= 7993 , step cost= 3 , total_cost= 36222 

step 7994
[[5 8 1]
 [7 0 4]
 [2 6 3]]
action= right , depth= 7994 , step cost= 4 , total_cost= 36226 

step 7995
[[5 8 1]
 [7 6 4]
 [2 0 3]]
action= up , depth= 7995 , step cost= 6 , total_cost= 36232 

step 7996
[[5 8 1]
 [7 6 4]
 [2 3 0]]
action= left , depth= 7996 , step cost= 3 , total_cost= 36235 

step 7997
[[5 8 1]
 [7 6 0]
 [2 3 4]]
action= down , depth= 7997 , step cost= 4 , total_cost= 36239 

step 7998

action= left , depth= 8158 , step cost= 4 , total_cost= 37014 

step 8159
[[8 1 2]
 [3 5 0]
 [6 4 7]]
action= down , depth= 8159 , step cost= 7 , total_cost= 37021 

step 8160
[[8 1 2]
 [3 0 5]
 [6 4 7]]
action= right , depth= 8160 , step cost= 5 , total_cost= 37026 

step 8161
[[8 1 2]
 [3 4 5]
 [6 0 7]]
action= up , depth= 8161 , step cost= 4 , total_cost= 37030 

step 8162
[[8 1 2]
 [3 4 5]
 [6 7 0]]
action= left , depth= 8162 , step cost= 7 , total_cost= 37037 

step 8163
[[8 1 2]
 [3 4 0]
 [6 7 5]]
action= down , depth= 8163 , step cost= 5 , total_cost= 37042 

step 8164
[[8 1 2]
 [3 0 4]
 [6 7 5]]
action= right , depth= 8164 , step cost= 4 , total_cost= 37046 

step 8165
[[8 1 2]
 [0 3 4]
 [6 7 5]]
action= right , depth= 8165 , step cost= 3 , total_cost= 37049 

step 8166
[[8 1 2]
 [6 3 4]
 [0 7 5]]
action= up , depth= 8166 , step cost= 6 , total_cost= 37055 

step 8167
[[8 1 2]
 [6 3 4]
 [7 0 5]]
action= left , depth= 8167 , step cost= 7 , total_cost= 37062 

step 8168
[[8 1 2]


step 8304
[[1 2 3]
 [8 0 4]
 [7 6 5]]
action= right , depth= 8304 , step cost= 4 , total_cost= 37813 

Time performance: 8364 nodes popped off the queue.
Space performance: 6503 nodes in the queue at its max.
Time spent: 23.16s


True

step 0
[[1 2 3]
 [8 6 4]
 [7 5 0]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[1 2 3]
 [8 6 4]
 [7 0 5]]
action= right , depth= 1 , step cost= 5 , total_cost= 5 

step 2
[[1 2 3]
 [8 6 4]
 [0 7 5]]
action= right , depth= 2 , step cost= 7 , total_cost= 12 

step 3
[[1 2 3]
 [0 6 4]
 [8 7 5]]
action= down , depth= 3 , step cost= 8 , total_cost= 20 

step 4
[[1 2 3]
 [6 0 4]
 [8 7 5]]
action= left , depth= 4 , step cost= 6 , total_cost= 26 

step 5
[[1 2 3]
 [6 4 0]
 [8 7 5]]
action= left , depth= 5 , step cost= 4 , total_cost= 30 

step 6
[[1 2 3]
 [6 4 5]
 [8 7 0]]
action= up , depth= 6 , step cost= 5 , total_cost= 35 

step 7
[[1 2 3]
 [6 4 5]
 [8 0 7]]
action= right , depth= 7 , step cost= 7 , total_cost= 42 

step 8
[[1 2 3]
 [6 4 5]
 [0 8 7]]
action= right , depth= 8 , step cost= 8 , total_cost= 50 

step 9
[[1 2 3]
 [0 4 5]
 [6 8 7]]
action= down , depth= 9 , step cost= 6 , total_cost= 56 

step 10
[[1 2 3]
 [4 0 5]
 [6 8 7]]
action= left , depth= 10 , step co

action= up , depth= 161 , step cost= 7 , total_cost= 826 

step 162
[[6 2 3]
 [8 7 1]
 [5 4 0]]
action= left , depth= 162 , step cost= 4 , total_cost= 830 

step 163
[[6 2 3]
 [8 7 0]
 [5 4 1]]
action= down , depth= 163 , step cost= 1 , total_cost= 831 

step 164
[[6 2 3]
 [8 0 7]
 [5 4 1]]
action= right , depth= 164 , step cost= 7 , total_cost= 838 

step 165
[[6 2 3]
 [0 8 7]
 [5 4 1]]
action= right , depth= 165 , step cost= 8 , total_cost= 846 

step 166
[[6 2 3]
 [5 8 7]
 [0 4 1]]
action= up , depth= 166 , step cost= 5 , total_cost= 851 

step 167
[[6 2 3]
 [5 8 7]
 [4 0 1]]
action= left , depth= 167 , step cost= 4 , total_cost= 855 

step 168
[[6 2 3]
 [5 8 7]
 [4 1 0]]
action= left , depth= 168 , step cost= 1 , total_cost= 856 

step 169
[[6 2 3]
 [5 8 0]
 [4 1 7]]
action= down , depth= 169 , step cost= 7 , total_cost= 863 

step 170
[[6 2 3]
 [5 0 8]
 [4 1 7]]
action= right , depth= 170 , step cost= 8 , total_cost= 871 

step 171
[[6 2 3]
 [5 1 8]
 [4 0 7]]
action= up , depth= 1

action= right , depth= 292 , step cost= 7 , total_cost= 1466 

step 293
[[2 3 8]
 [0 6 7]
 [4 5 1]]
action= right , depth= 293 , step cost= 6 , total_cost= 1472 

step 294
[[2 3 8]
 [4 6 7]
 [0 5 1]]
action= up , depth= 294 , step cost= 4 , total_cost= 1476 

step 295
[[2 3 8]
 [4 6 7]
 [5 0 1]]
action= left , depth= 295 , step cost= 5 , total_cost= 1481 

step 296
[[2 3 8]
 [4 6 7]
 [5 1 0]]
action= left , depth= 296 , step cost= 1 , total_cost= 1482 

step 297
[[2 3 8]
 [4 6 0]
 [5 1 7]]
action= down , depth= 297 , step cost= 7 , total_cost= 1489 

step 298
[[2 3 8]
 [4 0 6]
 [5 1 7]]
action= right , depth= 298 , step cost= 6 , total_cost= 1495 

step 299
[[2 3 8]
 [4 1 6]
 [5 0 7]]
action= up , depth= 299 , step cost= 1 , total_cost= 1496 

step 300
[[2 3 8]
 [4 1 6]
 [5 7 0]]
action= left , depth= 300 , step cost= 7 , total_cost= 1503 

step 301
[[2 3 8]
 [4 1 0]
 [5 7 6]]
action= down , depth= 301 , step cost= 6 , total_cost= 1509 

step 302
[[2 3 8]
 [4 0 1]
 [5 7 6]]
action= rig

step 411
[[3 8 5]
 [6 2 0]
 [7 1 4]]
action= down , depth= 411 , step cost= 4 , total_cost= 1976 

step 412
[[3 8 5]
 [6 0 2]
 [7 1 4]]
action= right , depth= 412 , step cost= 2 , total_cost= 1978 

step 413
[[3 8 5]
 [6 1 2]
 [7 0 4]]
action= up , depth= 413 , step cost= 1 , total_cost= 1979 

step 414
[[3 8 5]
 [6 1 2]
 [7 4 0]]
action= left , depth= 414 , step cost= 4 , total_cost= 1983 

step 415
[[3 8 5]
 [6 1 0]
 [7 4 2]]
action= down , depth= 415 , step cost= 2 , total_cost= 1985 

step 416
[[3 8 5]
 [6 0 1]
 [7 4 2]]
action= right , depth= 416 , step cost= 1 , total_cost= 1986 

step 417
[[3 8 5]
 [6 4 1]
 [7 0 2]]
action= up , depth= 417 , step cost= 4 , total_cost= 1990 

step 418
[[3 8 5]
 [6 4 1]
 [7 2 0]]
action= left , depth= 418 , step cost= 2 , total_cost= 1992 

step 419
[[3 8 5]
 [6 4 0]
 [7 2 1]]
action= down , depth= 419 , step cost= 1 , total_cost= 1993 

step 420
[[3 8 5]
 [6 0 4]
 [7 2 1]]
action= right , depth= 420 , step cost= 4 , total_cost= 1997 

step 421
[[

step 536
[[8 5 7]
 [4 6 2]
 [0 1 3]]
action= right , depth= 536 , step cost= 1 , total_cost= 2471 

step 537
[[8 5 7]
 [0 6 2]
 [4 1 3]]
action= down , depth= 537 , step cost= 4 , total_cost= 2475 

step 538
[[8 5 7]
 [6 0 2]
 [4 1 3]]
action= left , depth= 538 , step cost= 6 , total_cost= 2481 

step 539
[[8 5 7]
 [6 2 0]
 [4 1 3]]
action= left , depth= 539 , step cost= 2 , total_cost= 2483 

step 540
[[8 5 7]
 [6 2 3]
 [4 1 0]]
action= up , depth= 540 , step cost= 3 , total_cost= 2486 

step 541
[[8 5 7]
 [6 2 3]
 [4 0 1]]
action= right , depth= 541 , step cost= 1 , total_cost= 2487 

step 542
[[8 5 7]
 [6 2 3]
 [0 4 1]]
action= right , depth= 542 , step cost= 4 , total_cost= 2491 

step 543
[[8 5 7]
 [0 2 3]
 [6 4 1]]
action= down , depth= 543 , step cost= 6 , total_cost= 2497 

step 544
[[8 5 7]
 [2 0 3]
 [6 4 1]]
action= left , depth= 544 , step cost= 2 , total_cost= 2499 

step 545
[[8 5 7]
 [2 3 0]
 [6 4 1]]
action= left , depth= 545 , step cost= 3 , total_cost= 2502 

step 546


...

action= left , depth= 7931 , step cost= 6 , total_cost= 35952 

step 7932
[[5 8 1]
 [3 6 4]
 [7 2 0]]
action= up , depth= 7932 , step cost= 4 , total_cost= 35956 

step 7933
[[5 8 1]
 [3 6 4]
 [7 0 2]]
action= right , depth= 7933 , step cost= 2 , total_cost= 35958 

step 7934
[[5 8 1]
 [3 6 4]
 [0 7 2]]
action= right , depth= 7934 , step cost= 7 , total_cost= 35965 

step 7935
[[5 8 1]
 [0 6 4]
 [3 7 2]]
action= down , depth= 7935 , step cost= 3 , total_cost= 35968 

step 7936
[[5 8 1]
 [6 0 4]
 [3 7 2]]
action= left , depth= 7936 , step cost= 6 , total_cost= 35974 

step 7937
[[5 8 1]
 [6 4 0]
 [3 7 2]]
action= left , depth= 7937 , step cost= 4 , total_cost= 35978 

step 7938
[[5 8 1]
 [6 4 2]
 [3 7 0]]
action= up , depth= 7938 , step cost= 2 , total_cost= 35980 

step 7939
[[5 8 1]
 [6 4 2]
 [3 0 7]]
action= right , depth= 7939 , step cost= 7 , total_cost= 35987 

step 7940
[[5 8 1]
 [6 4 2]
 [0 3 7]]
action= right , depth= 7940 , step cost= 3 , total_cost= 35990 

step 7941
[[5 8 1]

action= up , depth= 8065 , step cost= 2 , total_cost= 36537 

step 8066
[[8 1 2]
 [5 3 7]
 [4 6 0]]
action= up , depth= 8066 , step cost= 7 , total_cost= 36544 

step 8067
[[8 1 2]
 [5 3 7]
 [4 0 6]]
action= right , depth= 8067 , step cost= 6 , total_cost= 36550 

step 8068
[[8 1 2]
 [5 3 7]
 [0 4 6]]
action= right , depth= 8068 , step cost= 4 , total_cost= 36554 

step 8069
[[8 1 2]
 [0 3 7]
 [5 4 6]]
action= down , depth= 8069 , step cost= 5 , total_cost= 36559 

step 8070
[[8 1 2]
 [3 0 7]
 [5 4 6]]
action= left , depth= 8070 , step cost= 3 , total_cost= 36562 

step 8071
[[8 1 2]
 [3 7 0]
 [5 4 6]]
action= left , depth= 8071 , step cost= 7 , total_cost= 36569 

step 8072
[[8 1 2]
 [3 7 6]
 [5 4 0]]
action= up , depth= 8072 , step cost= 6 , total_cost= 36575 

step 8073
[[8 1 2]
 [3 7 6]
 [5 0 4]]
action= right , depth= 8073 , step cost= 4 , total_cost= 36579 

step 8074
[[8 1 2]
 [3 7 6]
 [0 5 4]]
action= right , depth= 8074 , step cost= 5 , total_cost= 36584 

step 8075
[[8 1 2]
 

action= up , depth= 8181 , step cost= 3 , total_cost= 37125 

step 8182
[[8 1 2]
 [7 3 6]
 [4 5 0]]
action= left , depth= 8182 , step cost= 5 , total_cost= 37130 

step 8183
[[8 1 2]
 [7 3 0]
 [4 5 6]]
action= down , depth= 8183 , step cost= 6 , total_cost= 37136 

step 8184
[[8 1 2]
 [7 0 3]
 [4 5 6]]
action= right , depth= 8184 , step cost= 3 , total_cost= 37139 

step 8185
[[8 1 2]
 [7 5 3]
 [4 0 6]]
action= up , depth= 8185 , step cost= 5 , total_cost= 37144 

step 8186
[[8 1 2]
 [7 5 3]
 [4 6 0]]
action= left , depth= 8186 , step cost= 6 , total_cost= 37150 

step 8187
[[8 1 2]
 [7 5 0]
 [4 6 3]]
action= down , depth= 8187 , step cost= 3 , total_cost= 37153 

step 8188
[[8 1 2]
 [7 0 5]
 [4 6 3]]
action= right , depth= 8188 , step cost= 5 , total_cost= 37158 

step 8189
[[8 1 2]
 [0 7 5]
 [4 6 3]]
action= right , depth= 8189 , step cost= 7 , total_cost= 37165 

step 8190
[[8 1 2]
 [4 7 5]
 [0 6 3]]
action= up , depth= 8190 , step cost= 4 , total_cost= 37169 

step 8191
[[8 1 2]
 [

 [7 6 5]]
action= down , depth= 8303 , step cost= 5 , total_cost= 37809 

step 8304
[[1 2 3]
 [8 0 4]
 [7 6 5]]
action= right , depth= 8304 , step cost= 4 , total_cost= 37813 

Time performance: 8364 nodes popped off the queue.
Space performance: 6503 nodes in the queue at its max.
Time spent: 27.47s


True